# Eligibility Trace: On-policy, Expected Sarsa Lambda, Stochastic

In [1]:
import numpy as np

## Create environment

In [2]:
def create_environment_states():
    """Creates environment states.

    Returns:
        num_states: int, number of states.
        num_terminal_states: int, number of terminal states.
        num_non_terminal_states: int, number of non terminal states.
    """
    num_states = 16
    num_terminal_states = 2
    num_non_terminal_states = num_states - num_terminal_states

    return num_states, num_terminal_states, num_non_terminal_states

In [3]:
def create_environment_actions(num_non_terminal_states):
    """Creates environment actions.

    Args:
        num_non_terminal_states: int, number of non terminal states.

    Returns:
        max_num_actions: int, max number of actions possible.
        num_actions_per_non_terminal_state: array[int], number of actions per
            non terminal state.
    """
    max_num_actions = 4

    num_actions_per_non_terminal_state = np.repeat(
        a=max_num_actions, repeats=num_non_terminal_states)

    return max_num_actions, num_actions_per_non_terminal_state

In [4]:
def create_environment_successor_counts(num_states, max_num_actions):
    """Creates environment successor counts.

    Args:
        num_states: int, number of states.
        max_num_actions: int, max number of actions possible.
    Returns:
        num_sp: array[int], number of successor
            states s' that can be reached from state s by taking action a.
    """
    num_sp = np.repeat(
        a=1, repeats=num_states * max_num_actions)

    num_sp = np.reshape(
        a=num_sp,
        newshape=(num_states, max_num_actions))

    return num_sp

In [5]:
def create_environment_successor_arrays(
        num_non_terminal_states, max_num_actions):
    """Creates environment successor arrays.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
    Returns:
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
    """
    sp_idx = np.array(
        object=[1, 0, 14, 4,
                2, 1, 0, 5,
                2, 2, 1, 6,
                4, 14, 3, 7,
                5, 0, 3, 8,
                6, 1, 4, 9,
                6, 2, 5, 10,
                8, 3, 7, 11,
                9, 4, 7, 12,
                10, 5, 8, 13,
                10, 6, 9, 15,
                12, 7, 11, 11,
                13, 8, 11, 12,
                15, 9, 12, 13],
        dtype=np.int64)

    p = np.repeat(
        a=1.0, repeats=num_non_terminal_states * max_num_actions * 1)

    r = np.repeat(
        a=-1.0, repeats=num_non_terminal_states * max_num_actions * 1)

    sp_idx = np.reshape(
        a=sp_idx,
        newshape=(num_non_terminal_states, max_num_actions, 1))
    p = np.reshape(
        a=p,
        newshape=(num_non_terminal_states, max_num_actions, 1))
    r = np.reshape(
        a=r,
        newshape=(num_non_terminal_states, max_num_actions, 1))

    return sp_idx, p, r

In [6]:
def create_environment():
    """Creates environment.

    Returns:
        num_states: int, number of states.
        num_terminal_states: int, number of terminal states.
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        num_actions_per_non_terminal_state: array[int], number of actions per
            non terminal state.
        num_sp: array[int], number of successor
            states s' that can be reached from state s by taking action a.
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
    """
    (num_states,
     num_terminal_states,
     num_non_terminal_states) = create_environment_states()

    (max_num_actions,
     num_actions_per_non_terminal_state) = create_environment_actions(
        num_non_terminal_states)

    num_sp = create_environment_successor_counts(
        num_states, max_num_actions)

    (sp_idx,
     p,
     r) = create_environment_successor_arrays(
        num_non_terminal_states, max_num_actions)

    return (num_states,
            num_terminal_states,
            num_non_terminal_states,
            max_num_actions,
            num_actions_per_non_terminal_state,
            num_sp,
            sp_idx,
            p,
            r)

## Set hyperparameters

In [7]:
def set_hyperparameters():
    """Sets hyperparameters.

    Returns:
        num_episodes: int, number of episodes to train over.
        maximum_episode_length: int, max number of timesteps for an episode.
        alpha: float, alpha > 0, learning rate.
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
        trace_decay_lambda: float, trace decay parameter lambda.
        trace_update_type: int, trace update type, 0 = accumulating,
            1 = replacing.
    """
    num_episodes = 10000
    maximum_episode_length = 200
    alpha = 0.1
    epsilon = 0.1
    gamma = 1.0
    trace_decay_lambda = 0.9
    trace_update_type = 0

    return (num_episodes,
            maximum_episode_length,
            alpha,
            epsilon,
            gamma,
            trace_decay_lambda,
            trace_update_type)

## Create value function and policy arrays

In [8]:
def create_value_function_arrays(num_states, max_num_actions):
    """Creates value function arrays.

    Args:
        num_states: int, number of states.
        max_num_actions: int, max number of actions possible.
    Returns:
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
    """
    return np.zeros(shape=[num_states, max_num_actions], dtype=np.float64)

In [9]:
def create_policy_arrays(num_non_terminal_states, max_num_actions):
    """Creates policy arrays.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
    Returns:
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    policy = np.repeat(
        a=1.0 / max_num_actions,
        repeats=num_non_terminal_states * max_num_actions)

    policy = np.reshape(
        a=policy,
        newshape=(num_non_terminal_states, max_num_actions))

    return policy

## Create eligibility traces

In [10]:
def create_eligibility_traces(num_states, max_num_actions):
    """Creates eligibility trace arrays.

    Args:
        num_states: int, number of states.
        max_num_actions: int, max number of actions possible.
    Returns:
        eligibility_trace: array[float], keeps track of the eligibility the
            trace for each state-action pair Q(s, a).
    """
    return np.zeros(shape=[num_states, max_num_actions], dtype=np.float64)

## Create algorithm

In [11]:
# Set random seed so that everything is reproducible
np.random.seed(seed=0)

In [12]:
def initialize_epsiode(num_non_terminal_states, eligibility_trace):
    """Initializes epsiode with initial state and initial action.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        eligibility_trace: array[float], keeps track of the eligibility the
            trace for each state-action pair Q(s, a).
    Returns:
        init_s_idx: int, initial state index from set of non terminal states.
        eligibility_trace: array[float], keeps track of the eligibility the
            trace for each state-action pair Q(s, a).
    """
    # Reset eligibility traces for new episode
    eligibility_trace = np.zeros_like(a=eligibility_trace)
    
    # Randomly choose an initial state from all non-terminal states
    init_s_idx = np.random.randint(
        low=0, high=num_non_terminal_states, dtype=np.int64)

    return init_s_idx, eligibility_trace

In [13]:
def epsilon_greedy_policy_from_state_action_function(
        max_num_actions, q, epsilon, s_idx, policy):
    """Create epsilon-greedy policy from state-action value function.

    Args:
        max_num_actions: int, max number of actions possible.
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        s_idx: int, current state index.
        policy: array[float], learned stochastic policy of which action a to
            take in state s.
    Returns:
        policy: array[float], learned stochastic policy of which action a to
            take in state s.
    """
    # Save max state-action value and find the number of actions that have the
    # same max state-action value
    max_action_value = np.max(a=q[s_idx, :])
    max_action_count = np.count_nonzero(a=q[s_idx, :] == max_action_value)

    # Apportion policy probability across ties equally for state-action pairs
    # that have the same value and zero otherwise
    if max_action_count == max_num_actions:
        max_policy_prob_per_action = 1.0 / max_action_count
        remain_prob_per_action = 0.0
    else:
        max_policy_prob_per_action = (1.0 - epsilon) / max_action_count
        remain_prob_per_action = epsilon / (max_num_actions - max_action_count)

    policy[s_idx, :] = np.where(
        q[s_idx, :] == max_action_value,
        max_policy_prob_per_action,
        remain_prob_per_action)

    return policy

In [14]:
def loop_through_episode(
        num_non_terminal_states,
        max_num_actions,
        num_sp,
        sp_idx,
        p,
        r,
        q,
        policy,
        eligibility_trace,
        alpha,
        epsilon,
        gamma,
        trace_decay_lambda,
        trace_update_type,
        maximum_episode_length,
        s_idx):
    """Loops through episode to iteratively update policy.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        num_sp: array[int], number of successor states s' that can be reached
            from state s by taking action a.
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
        eligibility_trace: array[float], keeps track of the eligibility the
            trace for each state-action pair Q(s, a).
        alpha: float, alpha > 0, learning rate.
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
        trace_decay_lambda: float, trace decay parameter lambda.
        trace_update_type: int, trace update type, 0 = accumulating,
            1 = replacing.
        maximum_episode_length: int, max number of timesteps for an episode.
        s_idx: int, initial state index from set of non terminal states.
    Returns:
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    # Loop through episode steps until termination
    for t in range(0, maximum_episode_length):
        # Choose policy for chosen state by epsilon-greedy choosing from the
        # state-action-value function */
        policy = epsilon_greedy_policy_from_state_action_function(
            max_num_actions, q, epsilon, s_idx, policy)

        # Get action
        a_idx = np.random.choice(a=max_num_actions, p=policy[s_idx, :])

        # Get reward
        successor_state_transition_index = np.random.choice(
            a=num_sp[s_idx, a_idx],
            p=p[s_idx, a_idx, :])

        # Get reward from state and action
        reward = r[s_idx, a_idx, successor_state_transition_index]

        # Get next state
        next_s_idx = sp_idx[s_idx, a_idx, successor_state_transition_index]

        # Check to see if we actioned into a terminal state
        if next_s_idx >= num_non_terminal_states:
            # Calculate TD error delta
            delta = reward - q[s_idx, a_idx]

            # Update eligibility traces and state action value function with
            # TD error
            eligibility_trace, q = update_eligibility_trace_and_q(
                s_idx,
                a_idx,
                delta,
                num_non_terminal_states,
                max_num_actions,
                alpha,
                gamma,
                trace_decay_lambda,
                trace_update_type,
                q,
                eligibility_trace)

            break  # episode terminated since we ended up in a terminal state
        else:
            # Get next action, using expectation value
            expected_value_on_policy = np.sum(
                a=policy[next_s_idx, :] * q[next_s_idx, :])

            # Calculate TD error delta
            delta = reward + gamma * expected_value_on_policy - q[s_idx, a_idx]

            # Update eligibility traces and state action value function with
            # TD error
            eligibility_trace, q = update_eligibility_trace_and_q(
                s_idx,
                a_idx,
                delta,
                num_non_terminal_states,
                max_num_actions,
                alpha,
                gamma,
                trace_decay_lambda,
                trace_update_type,
                q,
                eligibility_trace)

            # Update state and action to next state and action
            s_idx = next_s_idx

    return q, policy

In [15]:
# This function updates the eligibility trace and state-action-value function
def update_eligibility_trace_and_q(
        s_idx,
        a_idx,
        delta,
        num_non_terminal_states,
        max_num_actions,
        alpha,
        gamma,
        trace_decay_lambda,
        trace_update_type,
        q,
        eligibility_trace):
    """Updates the eligibility trace and state-action-value function.

    Args:
        s_idx: int, initial state index from set of non terminal states.
        a_idx: int, initial action index from set of actions of state s_idx.
        delta: float, difference between estimated and target Q.
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        alpha: float, alpha > 0, learning rate.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
        trace_decay_lambda: float, trace decay parameter lambda.
        trace_update_type: int, trace update type, 0 = accumulating,
            1 = replacing.
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        eligibility_trace: array[float], keeps track of the eligibility the
            trace for each state-action pair Q(s, a).
    Returns:
        eligibility_trace: array[float], keeps track of the eligibility the
            trace for each state-action pair Q(s, a).
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
    """
    # Update eligibility trace
    if trace_update_type == 1:  # replacing
        eligibility_trace[s_idx, a_idx] = 1.0
    else:  # accumulating or unknown
        eligibility_trace[s_idx, a_idx] += 1.0

    # Update state-action-value function
    q += alpha * delta * eligibility_trace
    eligibility_trace *= gamma * trace_decay_lambda

    return eligibility_trace, q

In [16]:
def on_policy_stochastic_eligibility_trace_expected_sarsa_lambda(
        num_non_terminal_states,
        max_num_actions,
        num_sp,
        sp_idx,
        p,
        r,
        q,
        policy,
        eligibility_trace,
        alpha,
        epsilon,
        gamma,
        trace_decay_lambda,
        trace_update_type,
        maximum_episode_length,
        num_episodes):
    """Loops through episodes to iteratively update policy.

    Args:
        num_non_terminal_states: int, number of non terminal states.
        max_num_actions: int, max number of actions possible.
        num_sp: array[int], number of successor states s' that can be reached
            from state s by taking action a.
        sp_idx: array[int], state indices of new state s' of taking action a
            from state s.
        p: array[float], transition probability to go from state s to s' by
            taking action a.
        r: array[float], reward from new state s' from state s by taking
            action a.
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
        eligibility_trace: array[float], keeps track of the eligibility the
            trace for each state-action pair Q(s, a).
        alpha: float, alpha > 0, learning rate.
        epsilon: float, 0 <= epsilon <= 1, exploitation-exploration trade-off,
            higher means more exploration.
        gamma: float, 0 <= gamma <= 1, amount to discount future reward.
        trace_decay_lambda: float, trace decay parameter lambda.
        trace_update_type: int, trace update type, 0 = accumulating,
            1 = replacing.
        maximum_episode_length: int, max number of timesteps for an episode.
        num_episodes: int, number of episodes to train over.
    Returns:
        q: array[float], keeps track of the estimated value of each
            state-action pair Q(s, a).
        policy: array[float], learned stochastic policy of which
            action a to take in state s.
    """
    for episode in range(0, num_episodes):
        # Initialize episode to get initial state
        init_s_idx, eligibility_trace = initialize_epsiode(
            num_non_terminal_states, eligibility_trace)

        # Loop through episode and update the policy
        q, policy = loop_through_episode(
            num_non_terminal_states,
            max_num_actions,
            num_sp,
            sp_idx,
            p,
            r,
            q,
            policy,
            eligibility_trace,
            alpha,
            epsilon,
            gamma,
            trace_decay_lambda,
            trace_update_type,
            maximum_episode_length,
            init_s_idx)

    return q, policy

## Run algorithm

In [17]:
def run_algorithm():
    """Runs the algorithm."""
    (num_states,
     _,
     num_non_terminal_states,
     max_num_actions,
     _,
     num_sp,
     sp_idx,
     p,
     r) = create_environment()

    trace_decay_lambda = 0.9
    trace_update_type = 0
    (num_episodes,
     maximum_episode_length,
     alpha,
     epsilon,
     gamma,
     trace_decay_lambda,
     trace_update_type) = set_hyperparameters()

    q = create_value_function_arrays(num_states, max_num_actions)

    policy = create_policy_arrays(num_non_terminal_states, max_num_actions)
    
    eligibility_trace = create_eligibility_traces(num_states, max_num_actions)

    # Print initial arrays
    print("\nInitial state-action value function")
    print(q)

    print("\nInitial policy")
    print(policy)

    # Run on policy temporal difference sarsa
    q, policy = on_policy_stochastic_eligibility_trace_expected_sarsa_lambda(
        num_non_terminal_states,
        max_num_actions,
        num_sp,
        sp_idx,
        p,
        r,
        q,
        policy,
        eligibility_trace,
        alpha,
        epsilon,
        gamma,
        trace_decay_lambda,
        trace_update_type,
        maximum_episode_length,
        num_episodes)

    # Print final results
    print("\nFinal state-action value function")
    print(q)

    print("\nFinal policy")
    print(policy)

In [18]:
run_algorithm()


Initial state-action value function
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

Initial policy
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]

Final state-action value function
[[-3.36930478 -2.19619019 -1.         -3.33086265]
 [-4.38027994 -3.31830747 -2.19686878 -4.16745228]
 [-4.3897442  -4.19385815 -3.40334765 -3.38270328]
 [-3.33862362 -1.         -2.19511778 -3.37545725]
 [-4.30305083 -2.19652985 -2.19940086 -4.16851447]
 [-3.4002236  -3.38022168 -3.34388115 -3.34063848]
 [-3.38296766 -4.43888286 -4.33973086 -2.1